Importing the necessary Python libraries

# Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
#import rpy2

# Data Loading

In [3]:
train = pd.read_csv("cs-training.csv")
test = pd.read_csv("cs-test.csv")

In [4]:
test

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,NaN,0.885519,43,0,0.177513,5700.0,4,0,0,0,0.0
1,2,NaN,0.463295,57,0,0.527237,9141.0,15,0,4,0,2.0
2,3,NaN,0.043275,59,0,0.687648,5083.0,12,0,1,0,2.0
3,4,NaN,0.280308,38,1,0.925961,3200.0,7,0,2,0,0.0
4,5,NaN,1.000000,27,0,0.019917,3865.0,4,0,0,0,1.0
5,6,NaN,0.509791,63,0,0.342429,4140.0,4,0,0,0,1.0
6,7,NaN,0.587778,50,0,1048.000000,0.0,5,0,0,0,3.0
7,8,NaN,0.046149,79,1,0.369170,3301.0,8,0,1,0,1.0
8,9,NaN,0.013527,68,0,2024.000000,NaN,4,0,1,0,0.0
9,10,NaN,1.000000,23,98,0.000000,0.0,0,98,0,98,0.0


# Data Description

# Data Understanding

# Plan of Attack

# Data Cleansing

### Remove Whitespace

### NA Values

Replace with central tendency metric

### Outlier Values

# Exploratory Data Visualizations

# Data Standardization/Normalization

# Logistic Regression

### Multi-collinearity analysis

# Decision Tree

# Random Forest

# Support Vector Machines (SVM)

# Artificial Neural Networks (ANN)

# k-Nearest Neighbors (k-NN)

# Stacked Ensemble Model

# Comparison of All Models

# References